In [1]:
import numpy as np
import os
from astroquery.jplhorizons import Horizons

In [2]:
# Paths to directories containing data
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [3]:
# Constants
km2au = 1.496e8
c = 3e5

In [4]:
### Function to create a file in the 'Mikko' format required for shape modelling 

def mikkoWrite(filename,day_list,int_list,sunx,suny,sunz,earthx,earthy,earthz, min_dist):
# Code to write to lightcurve file in Mikko format
# Additional argument to limit files to points beyond min_dist from the sun

    counter = 0
    fout = open(filename, 'w+')
    sameday = []
    for i, day in enumerate(day_list[:-1]):
        if (np.sqrt(sunx[i]**2+suny[i]**2+sunz[i]**2) >= min_dist):
            newday = True
            sameday.append(day)
            if abs(day-day_list[i+1])<0.5:
                newday = False
            if newday==False:
                if day not in sameday:
                    sameday.append(day)
            else:
                fout.write('%i 1\n'%(len(sameday)))
                for time in sameday:
                    loc = day_list.index(time)
                    fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,int_list[loc],sunx[loc], suny[loc], sunz[loc], earthx[loc], earthy[loc], earthz[loc]))
                counter = counter+1
                sameday = []
    fout.write('1 1\n')
    fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(day_list[-1]+2400000.5,int_list[-1],sunx[-1], suny[-1], sunz[-1], earthx[-1], earthy[-1], earthz[-1]))
    counter = counter+1
    fout.close()
    return counter

In [5]:
# Read LSST sim output

lsst_file_name = path_67p+'LSST_out_LC_67P.txt'
lsst_file = open(lsst_file_name, 'r')

lsst_mjd =[]; lsst_delta=[]
lsst_ra = []; lsst_dec = []
lsst_sunx = []; lsst_suny = []; lsst_sunz = []
lsst_phase_ang = [];
lines_lsst = lsst_file.readlines()
for line in lines_lsst[25:]:
    dat = line.split()
    lsst_mjd.append(float(dat[2]))
    lsst_delta.append(float(dat[3]))
    lsst_ra.append(float(dat[5]))
    lsst_dec.append(float(dat[7]))
    lsst_sunx.append(float(dat[9]))
    lsst_suny.append(float(dat[10]))
    lsst_sunz.append(float(dat[11]))
    lsst_phase_ang.append(float(dat[21]))

lsst_delta = np.array(lsst_delta)/km2au
lsst_sunx = np.array(lsst_sunx)/km2au
lsst_suny = np.array(lsst_suny)/km2au
lsst_sunz = np.array(lsst_sunz)/km2au

In [6]:
# File taken from columns in OG spreadsheet WITH < 3AU OBS REMOVED**
# (just did this in an excel spreadsheet)
#lc_file = path_67p+'LSST_3AU_sun_comet.dat'

#mjd = []
#sun_x = []; sun_y = []; sun_z = []

#f = open(lc_file, 'r')
#lines = f.readlines()
#for line in lines:
#    data=line.split()
#    mjd.append(float(data[0]))
#    sun_x.append(float(data[2]))
#    sun_y.append(float(data[3]))
#    sun_z.append(float(data[4]))
#mjd[-1]+2400000.5

In [7]:
mjd = np.array(lsst_mjd)
sun_x = lsst_sunx
sun_y = lsst_suny
sun_z = lsst_sunz

- - - - - - - - - - - - -

In [8]:
# LSST output only has obj-Sun geom: also need Earth-obj
# Also want list of solar phase angles, to compute abs mag from phase function

# Splitting this into 2 because Horizons wouldn't let me do all 655 epochs at once :(
obj = Horizons(id='90000693', location='500',epochs=mjd[:500])

In [9]:
earth_x = []; earth_y=[]; earth_z=[];
vec = obj.vectors(refplane='earth')  # LSST output in equatorial ref frame!!!
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

/Users/s1523386/anaconda3/lib/python3.8/site-packages/astroquery/jplhorizons/core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


In [10]:
# Check output looks sensible
vec

targetname,datetime_jd,datetime_str,M1,M2,k1,k2,phasecoeff,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,mag,---,---,mag / deg,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str25,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67P/Churyumov-Gerasimenko,2459953.863937,A.D. 2023-Jan-09 08:44:04.1568,11.2,13.9,12.0,5.0,0.03,-3.444505659720326,-2.01529811172955,-0.7824777733431274,0.01361512061956163,-0.001640125884274181,-0.001023026664067215,0.02348749091349553,4.066732986656742,-0.01052233376238634
67P/Churyumov-Gerasimenko,2459956.868077,A.D. 2023-Jan-12 08:50:01.8528,11.2,13.9,12.0,5.0,0.03,-3.403978602868609,-2.019009305196668,-0.7850265415203742,0.01335816081538008,-0.0008328157860216082,-0.0006747539103341635,0.02330315049423218,4.034815432476373,-0.01072161335830465
67P/Churyumov-Gerasimenko,2459957.822871,A.D. 2023-Jan-13 07:44:56.0544,11.2,13.9,12.0,5.0,0.03,-3.391267457502642,-2.019683241381317,-0.7856184818099728,0.01326701767661173,-0.0005791162703836693,-0.0005652806996806055,0.02324386880187047,4.024551125628097,-0.01077841417826468
67P/Churyumov-Gerasimenko,2459964.842786,A.D. 2023-Jan-20 08:13:36.7104,11.2,13.9,12.0,5.0,0.03,-3.300839677733815,-2.017323136177724,-0.7868120079091985,0.01245369868918222,0.001235307693765872,0.0002185425969670557,0.02279992085942436,3.94768392220678,-0.01108792658197195
67P/Churyumov-Gerasimenko,2459966.871729,A.D. 2023-Jan-22 08:55:17.3856,11.2,13.9,12.0,5.0,0.03,-3.275855222497047,-2.014303930586379,-0.7861467618230418,0.0121708103463252,0.001738936136628179,0.000436405100539614,0.02266968158750547,3.925133691310624,-0.01113736255238601
67P/Churyumov-Gerasimenko,2459966.876288,A.D. 2023-Jan-22 09:01:51.2832,11.2,13.9,12.0,5.0,0.03,-3.275799737275682,-2.01429600022679,-0.7861447711491047,0.01217015096136869,0.001740054677198425,0.0004368890101460341,0.02266938833302884,3.925082915871988,-0.01113745147457623
67P/Churyumov-Gerasimenko,2459970.830448,A.D. 2023-Jan-26 07:55:50.7072,11.2,13.9,12.0,5.0,0.03,-3.228857768734552,-2.005529464626848,-0.7836014200889149,0.01156078626169196,0.002685687273778482,0.0008458551858015367,0.02241444519247415,3.880940879447828,-0.01117697335936618
67P/Churyumov-Gerasimenko,2459970.830859,A.D. 2023-Jan-26 07:56:26.2176,11.2,13.9,12.0,5.0,0.03,-3.228853017265195,-2.005528360789726,-0.7836010724339388,0.01156071923784386,0.002685782925363542,0.0008458965280223403,0.02241441866126721,3.880936285711749,-0.0111769736099083


In [11]:
# Now get the remaining 655-500 vectors
obj = Horizons(id='90000693', location='500', epochs=mjd[500:])

In [12]:
vec = obj.vectors(refplane='earth')  
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [13]:
# Check this didn't miss anything or double anything
#earth_x[495:]


In [14]:
# Light time correct the MJDs:
delta = lsst_delta
#for i in range(len(earth_x)):
#    delta.append(np.sqrt(earth_x[i]**2+earth_y[i]**2+earth_z[i]**2))
#delta=np.array(delta)
mjd_ltc = np.zeros_like(mjd)

mjd_ltc = mjd-(((delta*km2au)/c)/(3600*24))
mjd_ltc = np.ndarray.tolist(mjd_ltc)

In [15]:
for i,time in enumerate(mjd_ltc):
    print(time, mjd[i])

59953.34046686406 59953.363937
59956.34479107718 59956.368077
59957.29964427473 59957.322871
59964.320002923756 59964.342786
59966.3490760892 59966.371729
59966.353635384934 59966.376288
59970.30805011133 59970.330448
59970.30846113815 59970.330859
59970.323103093804 59970.3455
59970.323931147774 59970.346328
59972.297099372634 59972.319369
59973.35521357923 59973.377415
59977.31767781625 59977.339625
59978.30851600644 59978.3304
59979.344438877495 59979.366257
59980.278004965185 59980.299764
59984.24211719965 59984.263628
59984.245833433575 59984.267344
59984.25742616275 59984.278936
59984.25825021452 59984.27976
59984.30237198136 59984.323879
59990.31762575978 59990.338768
59990.333675713955 59990.354817
59990.349401646774 59990.370542
59991.25495492466 59991.276042
59991.271517907946 59991.292604
59992.31625583932 59992.337281
59992.33249678522 59992.353521
59994.31207227407 59994.332983
59994.31248729774 59994.333398
59994.32763415996 59994.348544
59994.32805218372 59994.348962
599

In [16]:
sun_xyz = np.array([sun_x, sun_y, sun_z])
print(sun_xyz[:,0])
earth_xyz = np.array([earth_x, earth_y, earth_z])
print(earth_xyz[:,0])

[-3.75625676 -1.15950396 -0.41150245]
[-3.44450566 -2.01529811 -0.78247777]


In [17]:
namestr = path_67p+'test.txt'
mikkoWrite(namestr,mjd_ltc,np.ones_like(mjd),sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

326

In [18]:
# Checking the phase angles: should make this into a function but too lazy
alpha_list_calc = []
for i in range(len(earth_xyz[1])):
    alpha_list_calc.append((360/(2.*np.pi))*np.arccos(np.dot(sun_xyz[:,i],earth_xyz[:,i])/(np.linalg.norm(sun_xyz[:,i])*np.linalg.norm(earth_xyz[:,i]))))

In [19]:
# This is just to grab ephemeris data to check phase angles
obj = Horizons(id='90000693', location='500', epochs=mjd[:500])
ephem_table = obj.ephemerides(quantities='43')

/Users/s1523386/anaconda3/lib/python3.8/site-packages/astroquery/jplhorizons/core.py:604: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


In [20]:
#ephem_table

In [21]:
alpha_list = []
for row in ephem_table['alpha_true']:
    alpha_list.append(float(row))

In [22]:
obj = Horizons(id='90000693', location='geocentric', epochs=mjd[500:])
ephem_table = obj.ephemerides()

In [23]:
for row in ephem_table['alpha_true']:
    alpha_list.append(float(row))

In [24]:
# Final step: read in intensities as calculated from shifts from Matlab script
# Obvs if you're using this script to create Mikko file in the first instance, you won't have these
intensities_file = path_67p+'67P_int_list_ALL.txt'
intensities_list = []
intf = open(intensities_file, 'r')
lines = intf.readlines()
for line in lines:
    data = line.split()
    intensities_list.append(float(data[0]))
intensities_list

[0.678042,
 0.82552,
 0.704318,
 0.808124,
 0.764408,
 0.764377,
 0.828405,
 0.828623,
 0.826144,
 0.825109,
 0.718381,
 0.760442,
 0.962517,
 0.92701,
 0.936208,
 0.782807,
 0.836797,
 0.843221,
 0.855684,
 0.856538,
 0.822295,
 0.750528,
 0.698892,
 0.710707,
 1.025672,
 1.023362,
 1.015963,
 0.975489,
 0.942462,
 0.943461,
 0.994464,
 0.996164,
 0.770433,
 0.76899,
 0.772581,
 0.773184,
 0.907094,
 0.935249,
 0.843734,
 0.872258,
 1.050144,
 1.133207,
 1.065174,
 0.964275,
 0.907414,
 0.946423,
 0.973848,
 1.291797,
 1.240163,
 1.026751,
 1.063382,
 1.12289,
 1.184097,
 1.185622,
 1.224697,
 1.228009,
 1.301182,
 1.307538,
 1.389966,
 1.451283,
 1.397427,
 1.451691,
 1.452701,
 1.528502,
 1.373747,
 1.371597,
 1.463118,
 1.613163,
 1.448603,
 1.499991,
 1.518339,
 1.566654,
 1.381562,
 1.337702,
 1.717459,
 1.781157,
 1.632861,
 1.38065,
 1.427394,
 1.519256,
 1.565231,
 1.361444,
 1.399792,
 1.259041,
 1.262976,
 1.566859,
 1.636984,
 1.27559,
 1.219316,
 0.821036,
 0.843811,
 0.82

In [25]:
# Also making standard lightcurve output with dates, magnitudes and 0.01 mag. uncertainty

corr_fac = 16    # from mag_from_phase_func py script

#fout = open(path_67p+'67p_20210109_I11_R.dat', 'w+')
for i,day in enumerate(mjd):
    magnitude = corr_fac -2.5*np.log10(intensities_list[i])
    #fout.write('%.6f %.4f 0.01\n'%(day+2400000.5, magnitude))
#fout.close()